## **Configurações Inciais**

In [ ]:
!pip install geemap

In [2]:
import ee
import geemap

In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
Map = geemap.Map()
Map

In [24]:
area = Map.user_roi
area.getInfo()

{'type': 'Polygon',
 'coordinates': [[[-41.835938, -8.146243],
   [-37.177734, -8.146243],
   [-37.177734, -1.669686],
   [-41.835938, -1.669686],
   [-41.835938, -8.146243]]]}

In [58]:
Map = geemap.Map(center=[-5.498398, -39.320624], zoom=7)

region = ee.Geometry.Point(area)

## **Mapa 1 Configurações**

In [70]:
def getImageSatelliteOne(year):
  start_date = ee.Date.fromYMD(year, 1, 1)
  end_date = start_date.advance(1, 'year')

  imagem_mapa = ee.ImageCollection("MODIS/061/MOD09A1")\
          .filterBounds(area)\
          .filterDate(start_date, end_date)\
          .first()

  return imagem_mapa

In [ ]:
anos = ee.List.sequence(2001, 2023)
imagens = anos.map(getImageSatelliteOne)
contar = imagens.size().getInfo()
lista_anos = anos.getInfo()
lista_anos

In [74]:
props_mapa1 = {
    'bands': ['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03'],
    'min': -100.0,
    'max': 3000.0,
}

In [ ]:
for index in range(0, contar):
    imagem = ee.Image(imagens.get(index))
    nome_camada = "Mapa(1) " + str(lista_anos[index])
    Map.addLayer(imagem, props_mapa1, 'Remote Sensing Reflectance', True, 0.8)
    Map.add_legend(title="Mapa GEOGRAFICO CEARÁ", builtin_legend='mapa1')
    #https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD09A1

In [ ]:
  #Map.addLayer(imagem_mapa, props_mapa1, 'Remote Sensing Reflectance', True, 0.8)
  #Map.add_legend(title="Mapa GEOGRAFICO CEARÁ", builtin_legend='mapa1')
  #https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD09A1

## **Mapa 2 Configurações**

In [126]:
def melhores_imagens(ano):
    ano_inicio = ee.Date.fromYMD(ano, 1, 1)
    ano_fim = ano_inicio.advance(1, 'year')

    imagem = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR") \
                .filterDate(ano_inicio, ano_fim) \
                .filterBounds(area_int) \
                .sort("CLOUD_COVER") \
                .first() \

    return imagem

In [ ]:
anos_map2 = ee.List.sequence(1984, 2011)
imagens_map2 = anos_map2.map(melhores_imagens)
contar_map2 = imagens_map2.size().getInfo()
print(contar_map2)
lista_anos_map2 = anos_map2.getInfo()
lista_anos_map2

In [129]:
props_map2 = {
    'bands': ['B3', 'B2', 'B1'],
    'min': 300,
    'max': 1000,
    'gamma': 1.0
}

In [130]:
for index_map2 in range(0,contar_map2):
  imagem_map2 = ee.Image(imagens_map2.get(index_map2))
  nome_camada_two = "Imagem(2) " + str(lista_anos[index])
  Map.addLayer(imagem_map2, props_map2, nome_camada_two)
  Map.add_legend(title="Mapa GEOGRAFICO CEARÁ 2", builtin_legend='mapa2')

In [ ]:
#Map.addLayer(imagem_mapa2, {}, 'NDVI')
#Map.add_legend(title="Mapa do Ceará 2", builtin_legend='mapa2')
#https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD13A2

In [ ]:
Map

## **Configurações de Inicialização**

In [61]:
left_layer = geemap.ee_tile_layer(imagem_mapa, props_mapa1, 'Remote Sensing Reflectance', True, 0.8)
right_layer = geemap.ee_tile_layer(imagem_mapa2, {}, 'NDVI')

In [63]:
Map.split_map(left_layer, right_layer)

In [ ]:
Map